# **Neural steering controller for autonomous parking**

This tutorial implements the physics-driven neural steering controller of the paper titled:

*Fast Planning and Tracking of Complex Autonomous Parking Maneuvers With Optimal Control and Pseudo-Neural Networks*

(available at https://ieeexplore.ieee.org/abstract/document/10309845)

## Initialization

### Import packages

In [8]:
import sys
import os
import torch
import pandas as pd
import scipy as sp

print('Current working directory: ',os.getcwd())
sys.path.append(os.path.join(os.getcwd(),'..'))
from neu4mes import *
from neu4mes import relation
from neu4mes import earlystopping
relation.NeuObj_names = []  # reset the list of NeuObj names

# import a library for plots
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
#mpl.rcParams['text.usetex'] = True
import matplotlib.pyplot as plt
plt.close('all')
SMALL_SIZE = 14
MEDIUM_SIZE = 22
BIGGER_SIZE = 26
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('grid', linestyle="--", color='grey')

# enable zooming on the plots
%matplotlib inline
import mpld3
mpld3.enable_notebook()


Current working directory:  /Users/mattiapiccinini/Documents/Research/Neu4Mes/Neu4Mes/tutorials


### Configurations, known constant parameters and initial guesses

In [9]:
# Path to the data folder
data_folder = os.path.join(os.getcwd(),'datasets','control_steer_car_parking')

# Import the file with the vehicle data
vehicle_data_csv = os.path.join(data_folder,'other_data','vehicle_data.csv')

# Import the file with the steering maps
steer_maps_file = os.path.join(data_folder,'other_data','steer_map.txt')
steer_map_load  = np.loadtxt(steer_maps_file, delimiter='\t', skiprows=1)
delta_w_avg_map_values = np.float64(np.deg2rad(steer_map_load[:,0]))
delta_w_avg_map_values = delta_w_avg_map_values[:,np.newaxis].tolist()  # add a dimension
delta_sw_map_values = np.float64(np.deg2rad(steer_map_load[:,1]))
delta_sw_map_values = delta_sw_map_values[:,np.newaxis].tolist()  # add a dimension

# Initial guesses
# Load the initial guesses for the curvature diagram approximation, computed in Matlab with a 2nd order optimizer
initial_guesses_curv_diagr = sp.io.loadmat(os.path.join(data_folder,'other_data','initial_guesses_curv_diagram','fit_curv_diagr_5th_ord_poly.mat'))
curv_diagram_params_guess = initial_guesses_curv_diagr['optim_params_poly'][0].astype('float64')

## NN model

### Custom parametric functions

In [10]:
# -------------------------------------------------
# Curvature diagram
# -------------------------------------------------
def curvat_diagram(curv,L,h_1,h_2,h_3):
  return torch.atan(h_1*curv + h_2*torch.pow(curv,3) + h_3*torch.pow(curv,5) + curv*L)

### Internal architecture

In [11]:
relation.NeuObj_names = []  # reset the list of NeuObj names

# Neural model inputs and outputs
curv         = Input('curv')          # [1/m] path curvature
steer        = Input('steer')         # [rad] steering wheel angle 
steer_target = Input('steer_target')  # [rad] steering wheel angle --> this is used only as a training target

num_samples_future_curv = 15   # number of samples in the future for the curvature prediction
num_samples_past_steer  = 15   # number of samples in the past for the steering wheel angle prediction

# Non-trainable parameters:
# Vehicle data
L = Constant('L',values=pd.read_csv(vehicle_data_csv)['L'][0].item())  # [m] wheelbase

# Trainable parameters:
# Curvature diagram parameters
h_1_guess = Parameter('h_1',values=[[curv_diagram_params_guess[0].item()]])  # initial guess
h_2_guess = Parameter('h_2',values=[[curv_diagram_params_guess[1].item()]])  # initial guess
h_3_guess = Parameter('h_3',values=[[curv_diagram_params_guess[2].item()]])  # initial guess

# Parametric function to learn the curvature diagram 
out_curv_diagr = ParamFun(curvat_diagram,
                          constants=[L],
                          parameters=[h_1_guess,h_2_guess,h_3_guess])(curv.sw([0,num_samples_future_curv]))

# FIR layer to weigh the future predictions of the curvature diagram
out_fir = Fir(parameter='fir_future_curv', parameter_init=init_exp, 
              parameter_init_params={'size_index':0, 'max_value':0.1, 'lambda':5, 'monotonicity':'decreasing'})(out_curv_diagr)   

# Parametric function to model the steering map (i.e., the relation between the average steering angle at the front wheels and the steering wheel angle)
delta_w_avg_map = Constant('delta_w_avg_map',values=delta_w_avg_map_values)  # [rad] average steering angle at the front wheels
delta_sw_map    = Constant('delta_sw_map',values=delta_sw_map_values)        # [rad] steering wheel angle
out_steer_map   = ParamFun(linear_interp, 
                           constants=[delta_w_avg_map,delta_sw_map],
                           map_over_batch=False)(out_fir)    # map_over_batch=True is used to apply the function to each sample in the batch

# FIR layer to weigh the past steering wheel angles computed by the NN (auto-regressive model)
out_arx = Fir(parameter='fir_auto_regression', parameter_init=init_exp, 
              parameter_init_params={'size_index':0, 'max_value':1e-3, 'lambda':5, 'monotonicity':'increasing'})(steer.sw([-num_samples_past_steer,0]))  

# Output of the neural model
out = Output('steering_angle', out_steer_map + out_arx)

### Neu4Mes framework

In [12]:
# Create a neu4mes model
path_folder_save      = os.path.join(os.getcwd(),'trained_models','steer_controller_park')    # folder to save the model
steer_controller_park = Neu4mes(visualizer=TextVisualizer(1),seed=12,workspace=path_folder_save,save_history=False)  #visualizer=MPLVisulizer()

# Add the neural model to the neu4mes structure and neuralization of the model
steer_controller_park.addModel('steer_ctrl',[out])
steer_controller_park.addMinimize('steer_error', 
                                  steer_target.next(),  # next means the first value in the "future"
                                  out, 
                                  loss_function='mse')
steer_controller_park.neuralizeModel()

================================ Neu4mes Model =================================
{'Constants': {'L': {'dim': 1, 'values': 2.6},
               'delta_sw_map': {'dim': 1,
                                'sw': 100,
                                'values': [[-12.566370614359172],
                                           [-12.317667228843606],
                                           [-12.068966471309635],
                                           [-11.820265825356046],
                                           [-11.571565179514732],
                                           [-11.322864540459207],
                                           [-11.074163920743521],
                                           [-10.825463293624303],
                                           [-10.576762669863411],
                                           [-10.328062048210148],
                                           [-10.079361428370472],
                                           [-9.83066081047353

## Training and validation datasets

In [13]:
# Load the training and the validation dataset
data_struct = ['curv',('steer','steer_target')]  # both steer and steer_target are read from the same column of the csv file
data_folder_train = os.path.join(data_folder,'training')
data_folder_valid = os.path.join(data_folder,'validation')
data_folder_test  = os.path.join(data_folder,'test')
steer_controller_park.loadData(name='training_set', source=data_folder_train, format=data_struct, skiplines=1)
steer_controller_park.loadData(name='validation_set', source=data_folder_valid, format=data_struct, skiplines=1)
steer_controller_park.loadData(name='test_set', source=data_folder_test, format=data_struct, skiplines=1)

# check the definition of the windows in the inputs and outputs
#samples_test_set = steer_controller_park.get_samples('training_set', index=100, window=1) 
#print(samples_test_set)

============================ Neu4mes Model Dataset =============================
Dataset Name:                 training_set
Number of files:              1
Total number of samples:      3092
Shape of steer:               (3092, 15, 1)
Shape of curv:                (3092, 15, 1)
Shape of steer_target:        (3092, 1, 1)
============================ Neu4mes Model Dataset =============================
Dataset Name:                 validation_set
Number of files:              1
Total number of samples:      832
Shape of steer:               (832, 15, 1)
Shape of curv:                (832, 15, 1)
Shape of steer_target:        (832, 1, 1)
============================ Neu4mes Model Dataset =============================
Dataset Name:                 test_set
Number of files:              1
Total number of samples:      691
Shape of steer:               (691, 15, 1)
Shape of curv:                (691, 15, 1)
Shape of steer_target:        (691, 1, 1)


## Training

### Train the NN in open-loop (no auto-regressive term)

In [14]:
num_epochs = 6000
batch_size = 100
learn_rate = 1e-3  # learning rate
early_stop_patience = 100
training_pars_open_loop = {'num_of_epochs':num_epochs, 
                           'val_batch_size':batch_size, 
                           'train_batch_size':batch_size, 
                           'lr':learn_rate}

train_result_open_loop = steer_controller_park.trainModel(train_dataset='training_set', validation_dataset='validation_set', 
                                                          training_params=training_pars_open_loop, optimizer='Adam', shuffle_data=True,
                                                          early_stopping=earlystopping.early_stop_patience,  
                                                          early_stopping_params={'patience':early_stop_patience,
                                                                                 'error':'steer_error'},
                                                          select_model=earlystopping.select_best_model)  

======================== Neu4mes Model Train Parameters ========================
models:                       ['steer_ctrl']
num of epochs:                6000
update per epochs:            30.92
update per epochs=(n_samples-batch_size)/batch_size+1
shuffle data:                 True
early stopping:               early_stop_patience
early stopping params:        {'error': 'steer_error', 'patience': 100}
train dataset:                training_set
 - num of samples:            3092
 - batch size:                100
 - unused samples:            0.0
unused samples=n_samples-update_per_epochs*batch_size
val dataset:                  validation_set
val {batch size, samples}:    {100, 832}
minimizers:                   {'steer_error': {'A': 'SamplePart41',
                                               'B': 'steering_angle',
                                               'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:         

In [15]:
# Print the trained NN parameters
steer_controller_park.neuralizeModel()

# Save the trained model
steer_controller_park.saveModel()

# Load the trained model
# steer_controller_park.loadModel() 

================================ Neu4mes Model =================================
{'Constants': {'L': {'dim': 1, 'values': 2.6},
               'delta_sw_map': {'dim': 1,
                                'sw': 100,
                                'values': [[-12.566370614359172],
                                           [-12.317667228843606],
                                           [-12.068966471309635],
                                           [-11.820265825356046],
                                           [-11.571565179514732],
                                           [-11.322864540459207],
                                           [-11.074163920743521],
                                           [-10.825463293624303],
                                           [-10.576762669863411],
                                           [-10.328062048210148],
                                           [-10.079361428370472],
                                           [-9.83066081047353